In [1]:
%pwd

'c:\\Users\\deven\\Desktop\\Medical Chatbot\\medical_chatbot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\deven\\Desktop\\Medical Chatbot\\medical_chatbot'

In [4]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [5]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [6]:
extracted_data=load_pdf_file(data='Data/')

In [29]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
  text_chunks=text_splitter.split_documents(extracted_data)
  return text_chunks
    

In [30]:
text_chunks=text_split(extracted_data)


In [9]:

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



c:\Users\deven\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [25]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4Bfm7F_UHJWXVhzupyfKq3hKHK11RWPCahLUxxsbXak1Xky91j9jgieSGFqnLdFP8UdrVu")

In [26]:
index_name = "medbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [31]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore.from_documents(documents = text_chunks,index_name=index_name,embedding=embeddings)

In [32]:
vector_store


In [33]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5,"k":4}
)

In [34]:
retrived_docs = retriever.invoke("what is acne ?")

In [35]:
retrived_docs

[Document(metadata={'page': 39.0, 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged 

In [36]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [37]:
import os
print(os.getenv("GROQ_API_KEY"))  # Should print "new_api_key_here"


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


In [38]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='The translation of "I love programming" to French is:\n\n"J\'adore le programmation."', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 55, 'total_tokens': 77, 'completion_time': 0.029333333, 'prompt_time': 0.004451961, 'queue_time': 0.049735237, 'total_time': 0.033785294}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-4c8496ad-4651-4fb2-a817-2729c7dcff90-0', usage_metadata={'input_tokens': 55, 'output_tokens': 22, 'total_tokens': 77})

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
  """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, say that you don't know.
Use three sentences maximum and keep the answer concise.

Context:
{context}
"""
)

prompt = ChatPromptTemplate.from_messages(
  [
    ("system",system_prompt),
    ("human", "{input}"),
  ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

In [46]:
response = rag_chain.invoke({"input": "What is stats"})

In [47]:
print(response["answer"])

I don't know.
